<a href="https://colab.research.google.com/github/kavyajeetbora/nlp_rag/blob/master/langchain_masterclass/03_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q langchain langchain_community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


In [63]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableSequence, RunnableParallel

In [4]:
if os.path.exists(".env"):
    os.remove(".env")

from google.colab import files
uploaded = files.upload()
if uploaded:
    if load_dotenv(".env"):
        print("Uploaded and Loaded Sucessfully")

Saving .env to .env
Uploaded and Loaded Sucessfully


## 1. Load LLM Model

In [5]:
model = ChatOpenAI(model='gpt-3.5-turbo-0125')
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7b875a80c4f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7b875a80c730>, root_client=<openai.OpenAI object at 0x7b8759270a00>, root_async_client=<openai.AsyncOpenAI object at 0x7b875a80c5b0>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))

## 2. Chains

Chains are sequences of calls that allow you to create complex workflows in LangChain by combining LLMs with other utilities in a specific order. This orchestration helps manage the execution and information flow between components. Chains are essential for building intricate applications with LangChain.

![](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*K_B_09VKpSl9Sgbwd7a7TA.png)

In [53]:
prompt_template = ChatPromptTemplate.from_messages(
    messages = [
        ("system", "You are a {subject} teacher. Answer only within 50 tokens"),
        ("human", "Tell me what is {topic} ?")
    ]
)

## Create a combined chain using a LangChain Expression Language (LCEL)
chain = prompt_template | model | StrOutputParser()

result = chain.invoke({"subject": "physics", "topic": "Newton 1st law"})
result

"Newton's First Law states that an object will remain at rest or in uniform motion unless acted upon by an external force."

## LCEL Chains under the hood

how chains uses series of `Runnables` under the hood

LCEL provides a more user-friendly and declarative way to define chains, but they are ultimately translated into RunnableSequences for execution.

Here are some important points to know about this relationship:

- RunnableSequence: Forms the foundation for executing chains. It defines a sequence of steps (Runnables) that are executed in order.
- Runnable: Represents a single step in a chain. It can be an LLM, a prompt template, an output parser, or any other function.
- LCEL: Provides syntactic sugar over RunnableSequences, making it easier to define chains with the | operator.
- Under the Hood: When an LCEL chain is invoked, it's converted into a RunnableSequence with the corresponding Runnables.
- Benefits: This abstraction makes LangChain chains more modular, flexible, and composable.

In [52]:
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

chain = RunnableSequence(
    first = format_prompt,
    middle=[invoke_model],
    last = parse_output
)

chain

RunnableLambda(lambda x: prompt_template.format_prompt(**x))
| RunnableLambda(lambda x: model.invoke(x.to_messages()))
| RunnableLambda(lambda x: x.content)

In [54]:
response = chain.invoke({"subject": "chemistry", "topic": "theory of relativity"})
response

'The theory of relativity, proposed by Albert Einstein, describes how time, space, and mass are connected. It has two main parts - the general theory of relativity and the special theory of relativity.'

## Extending the chain

We can extend the chain with more additional functions like creating some metadata for the output generated by the model like:

In [62]:
prompt_template = ChatPromptTemplate.from_messages(
    messages = [
        ("system", "You are a {subject} teacher. Answer only within 50 tokens"),
        ("human", "Tell me what is {topic} ?")
    ]
)

title_case = RunnableLambda(lambda x: x.title())
word_count = RunnableLambda(lambda x: f"content: {x}\n\nWord count: {len(x.split(' '))}")


chain_2 = prompt_template | model | StrOutputParser() | title_case | word_count
result = chain_2.invoke({"subject": "english", "topic": "shakespear"})
print(result)

content: Shakespeare Is Regarded As One Of The Greatest Playwrights And Poets In English Literature. His Works Include Famous Plays Such As "Romeo And Juliet," "Hamlet," And "Macbeth."

Word count: 27


# Types of Chains

![](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*f6p_BgKsmGOUGUV2pagGFg.png)

## Running Chains in Parallel

Chain:

1. Prompt_template -> Product
2. model -> list of feature
3. StrOutputParser
4. RunnableParallel -> Invoke two prompts in parallel
5. RunnableLambda -> combine the outputs



In [68]:
prompt_template = ChatPromptTemplate.from_messages(
    messages = [
        ("system", "you are a tech product reviewer."),
        ("human", "Please provide the list of features for {product}")
    ]
)

def analyse_pros_template(features):
    prompt_template = ChatPromptTemplate.from_messages(
        messages = [
            ('system', "You are an expert tech reviwer"),
            ("human", "Provide the list of pros from the given {features}")
        ]
    )

    return prompt_template.format_prompt(features=features)

def analyse_cons_template(features):
    prompt_template = ChatPromptTemplate.from_messages(
        messages = [
            ('system', "You are an expert tech reviwer"),
            ("human", "Provide the list of cons from the given {features}")
        ]
    )

    return prompt_template.format_prompt(features=features)


run_pros_chain = RunnableLambda(lambda x: analyse_pros_template(x)) | model | StrOutputParser()

run_cons_chain = RunnableLambda(lambda x: analyse_cons_template(x)) | model | StrOutputParser()

final_output = RunnableLambda(lambda x: print("Pros:\n",x["branches"]['pros'],"\n\nCons:\n",x['branches']['cons']))

In [69]:
product_review_chain = (prompt_template
                        | model
                        | StrOutputParser()
                        | RunnableParallel(branches = {"pros": run_pros_chain, "cons": run_cons_chain})
                        | final_output
)

In [71]:
product_review_chain.invoke({'product': "Apple Macbook"})

Pros:
 Thank you for providing the details about the Apple MacBook. Below is the list of pros based on the features you shared:

1. **Retina Display**: The high-resolution display with vibrant colors and sharp details provides an excellent viewing experience for tasks like graphic design, photo editing, and watching videos.

2. **Touch Bar and Touch ID**: The Touch Bar offers customizable shortcuts that can enhance productivity, and the Touch ID adds a convenient and secure method for logging in and authorizing purchases.

3. **Powerful Processor**: The latest Intel processors ensure fast performance and smooth multitasking, making it suitable for demanding tasks like video editing and 3D rendering.

4. **macOS Operating System**: The macOS operating system is known for its stability, ease of use, and seamless integration with other Apple devices, providing a cohesive ecosystem for users.

5. **Thin and Lightweight Design**: The sleek and portable design makes it easy to carry around, 

## Summary

**Parallel Execution:**
- **Efficiency:** Faster as tasks run simultaneously.
- **Resource Utilization:** Better use of multiple cores/threads.
- **Complexity:** More complex due to synchronization needs.

**Sequential Execution:**
- **Simplicity:** Easier to implement and debug.
- **Predictability:** More predictable flow.
- **Time-Consuming:** Slower as tasks run one after another.

**LangChain on a Single Machine:**
- **Multiple Cores:** Distributes tasks across cores for true parallelism.
- **Multithreading:** Uses threads on a single core for concurrent execution, improving performance for I/O-bound tasks.

**Local LLM Models:**
- **Resource Utilization:** Fully leverages local hardware resources.
- **Latency Reduction:** Reduces network latency, enhancing parallel execution efficiency.
- **Customization:** Allows for environment optimizations tailored to specific hardware and use cases.